# DFT Manager
#### Austin Zadoks, Alejandro Strachan
---

In [5]:
%run launcher.py

In [6]:
install_libs()
show_projects()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…